In [4]:
import tweepy

ModuleNotFoundError: No module named 'tweepy'

In [1]:
# Scrape imports
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
from datetime import datetime
import shutil

# Processing imports
import pandas as pd
import json
import regex as re
import nltk
from nltk.tokenize import RegexpTokenizer 

ModuleNotFoundError: No module named 'tweepy'

# This big cell defines all the functions that are used at the bottom.
Sorry

In [2]:
def initiate_stream(to_track):
    consumer_key = 'bmHm3j3uPr843UAxhAFkVWy5K'
    consumer_secret = 'cRc0dFKxP3fcBOy4W4dhZHE70BkMXVqgryFR3DcA7MWdIoJMGL'
    access_token = '1034804083515961344-xWIE6Mq6VynHUVSdMF2nmdXTxw3b7w'
    access_secret = 'EXOYf0OPG6ID8DCE0wdml4T0H4qmghne5l6nx1FBGYH0R'

    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
 
    api = tweepy.API(auth)

    class MyListener(StreamListener):
        def on_data(self, data):
            try:
                with open('scrape.json', 'a') as f:
                    f.write(data)
                    return True
            except BaseException as e:
                print("Error on_data: %s" % str(e))
            return True
 
        def on_error(self, status):
            print(status)
            return True
        
    twitter_stream = Stream(auth, MyListener())
    twitter_stream.filter(track=to_track)

def after_stream():
    now = datetime.now()
    if now.month < 10:
        month = '0'+str(now.month)
    else: month = str(now.month)
    if now.day < 10:
        day = '0'+str(now.day)
    else: day = str(now.day)
    if now.hour < 10:
        hour = '0'+str(now.hour)
    else: hour = str(now.hour)
    if now.minute < 10:
        minute = '0'+str(now.minute)
    else: minute = str(now.minute)
    if now.second < 10:
        second = '0'+str(now.second)
    else: second = str(now.second)

    now_str = str(now.year)+month+day+'_'+hour+minute+second
    dest = './tweepy_scrape_'+now_str+'.json'
    shutil.move('./scrape.json', dest)

    input_filepath = dest

    # Read the entire file into a python array
    with open(input_filepath, 'r') as f:
        data = f.readlines()

    jsons = []
    for ind in data:
        jsons.append(json.loads(ind))

    # Read into json
    with open('my_test_data.json', 'w') as outfile:
        json.dump(jsons, outfile)
    
    # Create a pandas series of the json data
    json_df = pd.read_json('my_test_data.json',typ='series')

    # Convert the series into list of dictionaries
    dict_list = []
    for row in json_df:
        temp_dict = {}
    
        # Basic tweet data: ID and timestamp
        temp_dict['id'] = row['id']
        temp_dict['created_at'] = row['created_at']
    
        try:
            temp_dict['url'] = row['entities']['urls'][0]['url']
        except:
            # Recreate url using username and tweet ID
            user = row['user']['screen_name']
            url = 'https://twitter.com/'+user+'/status/'+row['id_str']
            temp_dict['url'] = url
    
        # Tweet location data, if available
        temp_dict['geo'] = row['geo']
        temp_dict['coords'] = row['coordinates']
    
        # Retrieve tweet text
        try:
            temp_dict['text'] = row['extended_tweet']['full_text']
        except:
            temp_dict['text'] = row['text']
    
        # Try to retrieve image url, if available
        try:
            temp_dict['image_url'] = row['extended_tweet']['entities']['media'][0]['media_url']
        except:
            temp_dict['image_url'] = None
    
        # Get user data
        temp_dict['user_id'] = row['user']['id']
        temp_dict['user_name'] = row['user']['name']
        temp_dict['user_screen_name'] = row['user']['screen_name']
        temp_dict['user_location'] = row['user']['location']
        temp_dict['user_geoenabled'] = row['user']['geo_enabled']
    
        # Append temp dict to dict_list
        dict_list.append(temp_dict)
    
    # Create a dataframe from the list of dictionaries
    data = pd.DataFrame(dict_list,index=list(range(len(dict_list))))

    filename = 'processed_tweepy_'+now_str+'.csv'

    # Write to csv
    data.to_csv(filename)
    df = pd.read_csv(filename)     # Just checking that it worked
    return df

def twitter_in(df,to_track):
    #tokenize the tweets
    tokenize_tweets = RegexpTokenizer(r'\w+')
        
    #create a list of tokens
    texts = [tokenize_tweets.tokenize(value) for index, value in df.text.iteritems()]
        
    #nake new column for tokens
    df['token_text'] = texts
        
    closed_words = ['closed', 'flooded', 'inaccessible', 'road', 'stuck', 'flood', 'close', 
                    'drive', 'lane', 'street']
    
    # Get tracking words compatible and append to closed_words
    to_track = [tokenize_tweets.tokenize(value) for value in to_track]
    track = []
    for sublist in to_track:
        for item in sublist:
            track.append(item)
    track = [word.lower() for word in track]
    closed_words += track
    
    count_list = []
    for word_list in df.token_text:
        count = 0   
        for word in word_list:
            if word.lower() in closed_words:
                count += 1
            if word.lower() == 'rt':
                count -= 1
        count_list.append(count) #could add different amounts for certain words
    df['num_closed_words'] = count_list
        
       
    if df[df.num_closed_words > 1].shape[0] == 0:
        print('no relevant tweets')
    else: 
        return df[df.num_closed_words > 1]

# This cell starts the live scrape. Remember to kernel interrupt when you're done collecting.
## This is also where you define your search terms in `to_track`

In [3]:
to_track = ['#HurricaneTest','ZapplesauceZ']
initiate_stream(to_track)
#### KERNEL INTERRUPT !!!!!!

NameError: name 'OAuthHandler' is not defined

In [85]:
df = after_stream()
twitter_in(df,to_track)

,Unnamed: 0,coords,created_at,geo,id,image_url,text,url,user_geoenabled,user_id,user_location,user_name,user_screen_name,token_text,num_closed_words
2,2,NaN,Wed Oct 03 00:03:07 +0000 2018,NaN,1047275820732112896,NaN,#HurricaneTest ZapplesauceZ,https://twitter.com/aprilg091/status/104727582...,False,1034804083515961344,NaN,April,aprilg091,"[HurricaneTest, ZapplesauceZ]",2
3,3,NaN,Wed Oct 03 00:03:13 +0000 2018,NaN,1047275844891267077,NaN,ZapplesauceZ ZapplesauceZ ZapplesauceZ,https://twitter.com/aprilg091/status/104727584...,False,1034804083515961344,NaN,April,aprilg091,"[ZapplesauceZ, ZapplesauceZ, ZapplesauceZ]",3


In [86]:
df.head()

,Unnamed: 0,coords,created_at,geo,id,image_url,text,url,user_geoenabled,user_id,user_location,user_name,user_screen_name,token_text,num_closed_words
0,0,NaN,Wed Oct 03 00:02:45 +0000 2018,NaN,1047275727585005568,NaN,#HurricaneTest my test,https://twitter.com/aprilg091/status/104727572...,False,1034804083515961344,NaN,April,aprilg091,"[HurricaneTest, my, test]",1
1,1,NaN,Wed Oct 03 00:02:58 +0000 2018,NaN,1047275783029444608,NaN,#HurricaneTest my test 2,https://twitter.com/aprilg091/status/104727578...,False,1034804083515961344,NaN,April,aprilg091,"[HurricaneTest, my, test, 2]",1
2,2,NaN,Wed Oct 03 00:03:07 +0000 2018,NaN,1047275820732112896,NaN,#HurricaneTest ZapplesauceZ,https://twitter.com/aprilg091/status/104727582...,False,1034804083515961344,NaN,April,aprilg091,"[HurricaneTest, ZapplesauceZ]",2
3,3,NaN,Wed Oct 03 00:03:13 +0000 2018,NaN,1047275844891267077,NaN,ZapplesauceZ ZapplesauceZ ZapplesauceZ,https://twitter.com/aprilg091/status/104727584...,False,1034804083515961344,NaN,April,aprilg091,"[ZapplesauceZ, ZapplesauceZ, ZapplesauceZ]",3
